In [1]:
import pandas as pd
import numpy as np
import math
from ortools.linear_solver import pywraplp

In [2]:
#--------DATAFRAME-----------
df = pd.read_csv('Fifa23PlayersData.csv')
df.drop(['Known As', 'On Loan'], axis=1, inplace = True)
df['Type'] = pd.cut(df['Overall'], bins=[0, 64, 74, float('Inf')], labels=['Bronze', 'Silver', 'Gold'])
df.drop('Positions Played', axis=1, inplace= True)
df.rename(columns={'Best Position':'Position'}, inplace= True)
df.rename(columns={'Full Name':'Name'}, inplace= True)
df = df.drop_duplicates()

In [15]:
#añadimos la posición y la edad a los nombres repetidos
df['Uniques']=np.where(df['Name'].duplicated(keep=False), df['Name']+' '+df['Position'].astype(str) +' ' +df['Age'].astype(str),df['Name'])

In [16]:
#cambiamos el último valor repetido manualmente
df['Uniques'][12164] = 'James Brown'

C:\Users\jl_12\AppData\Local\Temp\ipykernel_10728\2968509139.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Uniques'][12164] = 'James Brown'


In [3]:
porteros = df[df['Position'] == 'GK']
latizq = df[df['Position'].isin(('LB', 'LWB'))]
latdrch = df[df['Position'].isin(('RB', 'RWB'))]
centrales = df[df['Position'] == 'CB']
mediocentros = df[df['Position'].isin(('CDM', 'CM', 'CAM'))]
medioizq = df[df['Position'].isin(('LW', 'LM'))]
mediodrch = df[df['Position'].isin(('RW', 'RM'))]
delanteros = df[df['Position'].isin(('CF', 'ST'))]

In [4]:
POR = porteros['Image Link'].to_numpy().tolist()
LI = latizq['Image Link'].to_numpy().tolist()
LD = latdrch['Image Link'].to_numpy().tolist()
DFC1 = centrales['Image Link'].to_numpy().tolist()
DFC2 = centrales['Image Link'].to_numpy().tolist()
MC1 = mediocentros['Image Link'].to_numpy().tolist()
MC2 = mediocentros['Image Link'].to_numpy().tolist()
MI = medioizq['Image Link'].to_numpy().tolist()
MD = mediodrch['Image Link'].to_numpy().tolist()
DC1 = delanteros['Image Link'].to_numpy().tolist()
DC2 = delanteros['Image Link'].to_numpy().tolist()

In [5]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [6]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')

In [7]:
# Create the variables x and y.
POR = solver.IntVar(0, solver.infinity(), 'POR')
LI = solver.IntVar(0, solver.infinity(), 'LI')
DFC1 = solver.IntVar(0, solver.infinity(), 'DFC1')
DFC2 = solver.IntVar(0, solver.infinity(), 'DFC2')
LD = solver.IntVar(0, solver.infinity(), 'LD')
MI = solver.IntVar(0, solver.infinity(), 'MI')
MC1 = solver.IntVar(0, solver.infinity(), 'MC1')
MC2 = solver.IntVar(0, solver.infinity(), 'MC2')
MD = solver.IntVar(0, solver.infinity(), 'MD')
DC1 = solver.IntVar(0, solver.infinity(), 'DC1')
DC2 = solver.IntVar(0, solver.infinity(), 'DC2')

In [8]:
# Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0,1)
ct.SetCoefficient(POR, 1)
ct.SetCoefficient(LI, 1)
ct.SetCoefficient(DFC1, 1)
ct.SetCoefficient(DFC2, 1)
ct.SetCoefficient(LD, 1)
ct.SetCoefficient(MI, 1)
ct.SetCoefficient(MC1, 1)
ct.SetCoefficient(MC2, 1)
ct.SetCoefficient(MD, 1)
ct.SetCoefficient(DC1, 1)
ct.SetCoefficient(DC2, 1)



print('Number of constraints =', solver.NumConstraints())

Number of constraints = 1


In [9]:
# Create the objective function,  x + y.
objective = solver.Objective()
objective.SetCoefficient(POR, 1)
objective.SetCoefficient(LI, 1)
objective.SetCoefficient(DFC1, 1)
objective.SetCoefficient(DFC2, 1)
objective.SetCoefficient(LD, 1)
objective.SetCoefficient(MI, 1)
objective.SetCoefficient(MC1, 1)
objective.SetCoefficient(MC2, 1)
objective.SetCoefficient(MD, 1)
objective.SetCoefficient(DC1, 1)
objective.SetCoefficient(DC2, 1)
objective.SetMaximization()

In [10]:
solver.Solve()
print('Solution:')
print('Objective value =', objective.Value())
print('POR =', POR.solution_value())
print('LI =', LI.solution_value())
print('DFC1 =', DFC1.solution_value())
print('DFC2 =', DFC2.solution_value())
print('LD =', LD.solution_value())
print('MI =', MI.solution_value())
print('MC1 =', MC1.solution_value())
print('MC2 =', MC2.solution_value())
print('MD =', MD.solution_value())
print('DC1 =', DC1.solution_value())
print('DC2 =', DC2.solution_value())

Solution:
Objective value = 1.0
POR = 0.0
LI = 0.0
DFC1 = 0.0
DFC2 = 0.0
LD = 0.0
MI = 0.0
MC1 = 0.0
MC2 = 0.0
MD = 0.0
DC1 = 0.0
DC2 = 1.0


habría que pasar DATA PARA CADA UNO

In [13]:
# Datos en forma matricial
JUGADORES = df['Image Link'].to_numpy().tolist()

DATA = df[['Value(in Euro)', 'Overall', 'Potential']].to_numpy().tolist()

RESOURCES = [50000, 80 * 11, 85 * 11]

In [15]:
presupuesto = 5000

In [29]:
def solve_army(JUGADORES, DATA, RESOURCES):
  # Instanciamos solver
  solver = pywraplp.Solver('Minimize resource consumption', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # Creamos las variables a optimizar
  jugadores = [solver.IntVar(0, solver.infinity(), jugador) for jugador in JUGADORES]
  
  # Restricciones sobre la fuerza mínima de las tropas
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum((10 * DATA[u][-2] + DATA[u][-1]) * jugadores[u] for u, _ in enumerate(jugadores)) >= 10000)

  # Restricciones sobre los recursos máximos
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * jugadores[u] for u, _ in enumerate(jugadores)) <= RESOURCES[r])

  # Función objetivo (minimizar recursos consumidos)
  solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * jugadores[u] for u, _ in enumerate(jugadores)))

  # Resolvemos el problema
  status = solver.Solve()

  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solución =================')
    print(f'Resuelto en {solver.wall_time():.2f} milisegundos con {solver.iterations()} iteraciones')
    print()

    power = sum((10 * DATA[u][-2] + DATA[u][-1]) * jugadores[u].solution_value() for u, _ in enumerate(jugadores))
    print(f'Valor óptimo = {solver.Objective().Value()} 🌾🪵🪙recursos')
    print(f'Fuerza = 💪{power}')
    print('Tropas:')
    for u, _ in enumerate(jugadores):
      print(f' - {jugadores[u].name()} = {jugadores[u].solution_value()}')
    print()

    food = sum((DATA[u][0]) * jugadores[u].solution_value() for u, _ in enumerate(jugadores))
    wood = sum((DATA[u][1]) * jugadores[u].solution_value() for u, _ in enumerate(jugadores))
    gold = sum((DATA[u][2]) * jugadores[u].solution_value() for u, _ in enumerate(jugadores))
    print('Recursos:')
    print(f' - 🌾Comida = {food}')
    print(f' - 🪵Madera = {wood}')
    print(f' - 🪙Oro = {gold}')
  else:
      print('No se encontró una solución óptima.')

In [30]:
solve_army(JUGADORES, DATA, RESOURCES)

No se encontró una solución óptima.


In [125]:
df['Uniques']=np.where(df['Name'].duplicated(keep=False), df['Name']+' '+df['Position'].astype(str) +' ' +df['Age'].astype(str),df['Name'])

In [128]:
df['Uniques'].value_counts()

Lionel Messi       1
Matteo Anzolin     1
Arkadiusz Pyrka    1
Tobe Leysen        1
Sebastian Ring     1
                  ..
Alexander Prass    1
Mehdi Zeffane      1
Darwin Cerén       1
Nicolás Oroz       1
Nabin Rabha        1
Name: Uniques, Length: 18420, dtype: int64

In [127]:
df['Uniques'][12164] = 'James Brown'

C:\Users\jl_12\AppData\Local\Temp\ipykernel_12568\556856305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Uniques'][12164] = 'James Brown'


In [123]:
# BUSCAR UN JUGADOR

df[df['Uniques'].str.contains('James Brown', case=False)]

,Name,Overall,Potential,Value(in Euro),Position,Nationality,Image Link,Age,Height(in cm),Weight(in kg),...,RM Rating,LWB Rating,CDM Rating,RWB Rating,LB Rating,CB Rating,RB Rating,GK Rating,Type,Unique Name
12164,James Brown,63,68,675000,RWB,Malta,https://cdn.sofifa.net/players/236/027/23_60.png,24,185,79,...,57,63,61,63,64,64,64,19,Bronze,James Brown
12543,James Brown,63,68,675000,RWB,Republic of Ireland,https://cdn.sofifa.net/players/261/277/23_60.png,24,177,77,...,61,63,61,63,63,62,63,19,Bronze,James Brown RWB 24
